In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading dataset into pandas dataframe
data = pd.read_csv('train.csv')
print(data.head())

    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  
2             1.0   

In [3]:
print(data.isnull().sum())

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


In [4]:
# ลิสต์ค่าข้อมูลตัวเลข 0 1 2 3+ ในคอัลมน์ Dependents ว่ามีตัวเลขนี้กี่ตัว
data['Dependents'].value_counts()

Dependents
0     345
1     102
2     101
3+     51
Name: count, dtype: int64

In [5]:
# data.replace ทำการเข้าถึงฐานข้อมูลแล้ว ทำการเปลี่ยนค่าในฐานข้อมูล จาก '3+' แทนค่าเป็น '4' แล้วอัพเดทข้อมูลลงในตัวแปร data
data = data.replace(to_replace='3+', value='4')
data['Dependents'].value_counts()

Dependents
0    345
1    102
2    101
4     51
Name: count, dtype: int64

In [6]:
# เข้าถึงฐานข้อมูลแล้วใช้คำสั่ง dropna ทำการลบ เอาออกค่าในทุกแถวและคอลัมน์ที่ไม่มีค่า มักนำมาใช้กับข้อมูลที่ต้องการความชัดเจน ถูกต้อง
#** หลังจากพบเห็นค่าว่าง isnull จากโค้ด data.isnull().sum() ให้ใช้คำสั่ง dropna() ตาม
data = data.dropna()
print(data.isnull().sum())

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64


In [7]:
data.replace({'Loan_Status': {'N':0, 'Y':1}}, inplace=True)
print(data.head(8))

    Loan_ID Gender Married Dependents     Education Self_Employed  \
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   
5  LP001011   Male     Yes          2      Graduate           Yes   
6  LP001013   Male     Yes          0  Not Graduate            No   
7  LP001014   Male     Yes          4      Graduate            No   
8  LP001018   Male     Yes          2      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   
5             5417     

In [8]:
# คำสั่ง unique() ทำการlist ค่าที่อยู่ในแต่ล่ะคอลัมน์แต่ล่ะตัวออกมาโดยไม่ซํ้ากัน
print(data['Married'].unique())
print(data['Gender'].unique())
print(data['Self_Employed'].unique())
print(data['Property_Area'].unique())
print(data['Education'].unique())

['Yes' 'No']
['Male' 'Female']
['No' 'Yes']
['Rural' 'Urban' 'Semiurban']
['Graduate' 'Not Graduate']


In [9]:
data.replace({'Married': {'No':0, 'Yes':1},
                    'Gender': {'Male': 1, 'Female': 0},
                    'Self_Employed': {'No':0, 'Yes': 1},
                    'Property_Area': {'Rural':0, 'Semiurban':1, 'Urban':2},
                    'Education': {'Graduate': 1, 'Not Graduate': 2}}, inplace=True)
data

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,1,1,1,1,0,4583,1508.0,128.0,360.0,1.0,0,0
2,LP001005,1,1,0,1,1,3000,0.0,66.0,360.0,1.0,2,1
3,LP001006,1,1,0,2,0,2583,2358.0,120.0,360.0,1.0,2,1
4,LP001008,1,0,0,1,0,6000,0.0,141.0,360.0,1.0,2,1
5,LP001011,1,1,2,1,1,5417,4196.0,267.0,360.0,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0,0,0,1,0,2900,0.0,71.0,360.0,1.0,0,1
610,LP002979,1,1,4,1,0,4106,0.0,40.0,180.0,1.0,0,1
611,LP002983,1,1,1,1,0,8072,240.0,253.0,360.0,1.0,2,1
612,LP002984,1,1,2,1,0,7583,0.0,187.0,360.0,1.0,2,1


In [10]:
# ใช้คำสั่ง drop ทำการdrop ทั้งชื่อหัวข้อและค่าในแนวตั้ง axis=1
x = data.drop(columns= ['Loan_ID', 'Loan_Status'], axis=1)
y = data['Loan_Status']
y

1      0
2      1
3      1
4      1
5      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 480, dtype: int64

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=.1, stratify=y, random_state=2)
print(x.shape, xtrain.shape, xtest.shape)

(480, 11) (432, 11) (48, 11)


In [12]:
print(xtest)

     Gender  Married Dependents  Education  Self_Employed  ApplicantIncome  \
368       1        1          1          1              0             6325   
74        1        1          4          1              1             5266   
135       1        1          4          1              0             4000   
53        1        1          2          1              0             4616   
96        0        1          0          1              0             2484   
388       1        1          0          1              0             2333   
345       1        1          0          1              0             2499   
8         1        1          2          1              0             4006   
549       1        1          0          1              0             2785   
99        1        1          0          1              0             1759   
49        0        0          0          1              0             4000   
513       1        1          0          1              0       

In [13]:
# concat เป็นการเอาข้อมูลหลายๆ Dataframe มารวมกันในรูปแบบ Tables
# ในส่วนนี้ xtest มีข้อมูลถึง Gender ถึง ApplicantIncome ส่วน Y มีข้อมูลแค่ Loan_Status
# จะเอาข้อมูล2Dataframe มารวมกันใช้คำสั่ง pd.concat([xtest,ytest], axis=1) ส่วน axis=1 ให้รวมข้อมูลในคอลัมน์แนวตั้ง
data = pd.concat([xtest,ytest], axis=1)
data.to_csv('test.txt', index=False)

In [14]:
clf = svm.SVC(kernel='linear', C=1)

In [15]:
#train the support vector machine
# โยนข้อมุล xtrain ytrain เข้าไปเรียนรู้ใน clf ที่เก็บข้อมูล SVC 
clf.fit(xtrain, ytrain)

: 

In [ ]:
# model evalution

# Accuracy score in 
x_train_pred = clf.predict(xtrain)
train_data_accuracy = accuracy_score(x_train_pred, ytrain)
print('Accuracy Score on Training data :', train_data_accuracy)

# Accuracy in test score
x_test_pred = clf.predict(xtest)
test_data_accuracy = accuracy_score(x_test_pred, ytest)
print('Accuracy Score on Testing data :', test_data_accuracy)

Accuracy Score on Training data : 0.7916666666666666
Accuracy Score on Testing data : 0.8333333333333334


In [ ]:
# mำการคำนวณหาประสิทธิภาพของโมเดล (Precision ความแม่นยำ) (Recall ความครอบคลุม)F1 (ความแม่นยำของการจำแนกประเภท) โดยนำข้อมูลการทำนายจาก x_test_pred และ ytest มาหาความแม่นยำย่อยๆต่อ 
from sklearn.metrics import precision_score, recall_score, f1_score

prec = precision_score(x_test_pred, ytest)
rec = recall_score(x_test_pred, ytest)
f1 = f1_score(x_test_pred, ytest)

print('Precision:', prec)
print('Recall:', rec)
print('F1:', f1)

Precision: 0.9393939393939394
Recall: 0.8378378378378378
F1: 0.8857142857142857


In [ ]:
# making prediction
# จัดเก็บข้อมูล 11ข้อมูลในรุปแบบ tuple ชุดข้อมูลที่แสดงตามลำดับและไม่สามารถเปลี่ยนแปลงข้อมูลนั้นได้
inputs = (1,1,0,2,0,1800,2934.0,93.0,360.0,0.0,2)

# Change input data to numpy array and reshape
# ทำการแปลงข้อมูลในตัวแปร inputs เป็น array NumPy = [1 1 0 2 0 1800 2934 ...] 
#ส่วน .reshape(1,-1) แล้วแปลงไปเป็น array 2 มิติที่มี 1 แถว 11คอลัมน์ (ซึ่งเหมาะกับการนำไปใช้กับ SVM) - อ่านเพิ่มเติมใน OneNote
input_as_np_array = np.array(inputs).reshape(1,-1)

# นำโมเดล clf ที่ได้ถูกฝึกสอนแล้วเอามาทำนายการให้สินเชื่อของแต่ล่ะบุคคลในตัวแปร input_as_np_array
prediction = clf.predict(input_as_np_array)
print(prediction)

# เชคว่าค่าใน Prediciton เป็น 0 หรือไม่
if (prediction[0] == 0):
    print('This person should not get loan')
else:
    print('This person is qualify for a loan')

[0]
This person should not get loan


In [ ]:
#saving our model
import joblib

joblib.dump(clf, 'clf_model.pkl')

['clf_model.pkl']